In [15]:
import pandas as pd
import numpy as np
import os
import sys
import re

## Utils

In [16]:
# Function to replace a substring in a string column
def process_values(value):
    if isinstance(value, str):
        value = re.sub(r'[\r\n]+', ' - ', value).strip('"\' \)\()')

    return np.nan if not value else value


# Clean columns content
def process_columns(column_name):
    # return column_name

    # BASIC
    column_name = re.sub(r'([A-Z]+)', r' \1', column_name.lower())
    column_name = re.sub(r'(\s+|\.)', r' ', column_name).strip()
    column_name = re.sub(r'(\s+)', r'_', column_name)

    # COUNTRY AND REGION
    column_name = re.sub(r'(iso[2-3])code', r'\1', column_name)
    column_name = re.sub(r'(country)(iso[2-3])', r'\1_\2', column_name)
    column_name = re.sub(r'(country)(region)', r'\2', column_name)
    column_name = re.sub(r'(?<=region)(?=type|name)', '_', column_name)
    column_name = re.sub(r'(?<=region)(?=id)', '_', column_name)
    column_name = re.sub(r'(?<=type)(?=id|name)', '_', column_name)

    column_name = column_name.replace('country_code', 'country_iso3')

    # DATE RELATED
    column_name = re.sub(r'(?<=last)(?=updated)', '_', column_name)

    return column_name

# FORMAT DF:
def process_df(df):
    # PROCESSING
    # STRING VALUES (might set to NaN)
    df = df.applymap(process_values)
    # Uniformize NaN values
    df = df.replace(['', 'nan'], np.nan)

    # Drop all columns having only NaN
    df = df.dropna(axis=1, how='all')

    # Replace column names using the format function
    df = df.rename(columns=process_columns)

    # Filter value if exists
    if ('value' in df.columns):
        df = df.dropna(subset=['value'])
    
    # Change year type to int
    if ('year' in df.columns):
        df['year'] = df['year'].astype(int)   

    return df

## Load and Process Data

### all data

In [17]:
df_iea = pd.read_csv("../../../data/_raw/iea/iea_api_eei.csv")
df_iea = process_df(df_iea)
df_iea.head()

,country,tab,flow,enduse,activity,unit,year,value
69,Australia,Freight transport,SHARE_COUNTRY_EM,Freight trains,NaN,%,2013,0.90
70,Australia,Freight transport,SHARE_COUNTRY_EM,Trucks,NaN,%,2013,8.65
71,Australia,Freight transport,SHARE_FOSSIL,Domestic freight ships,NaN,%,2013,88.52
72,Australia,Freight transport,SHARE_FOSSIL,Freight trains,NaN,%,2013,94.43
73,Australia,Freight transport,SHARE_FOSSIL,Trucks,NaN,%,2013,99.21


In [18]:
df_info = pd.read_csv("../../../data/_info/__INFO_UN_M49_en.csv")
df_info = process_df(df_info)
df_info.head()

,global_code,global_name,region_code,region_name,sub-region_code,sub-region_name,intermediate_region_code,intermediate_region_name,country_or_area,m49_code,iso-alpha2_code,iso-alpha3_code,least_developed_countries_(ldc),land_locked_developing_countries_(lldc),small_island_developing_states_(sids)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


In [19]:
# Mapping data with info_m49_en

# Left joining
df_merged = df_iea.merge(df_info, left_on='Country', right_on='Country or Area', how='left')

# Filter columns
selected_columns = ['Country', 'ISO-alpha3 Code', 'Year', 'Tab', 'Flow', 'enduse', 'region_name', 'sub-region_name', 'activity', 'unit', 'value']
df_merged = df_merged[selected_columns]
df_merged.head()

KeyError: 'Country or Area'

### Final energy

In [48]:
df_final_energy = df_merged[df_merged['flow']=='E_FINAL']
df_final_energy = df_final_energy.drop(columns=['activity'])
df_final_energy = df_final_energy.groupby(['country', 'iso-alpha3_code', 'year', 'tab', 'region_name', 'sub-region_name', 'unit'])['value'].sum().reset_index()
df_final_energy["source"] = 'iea'
df_final_energy["type"] = 'final_energy'
df_final_energy.rename(columns={'iso-alpha3_code':'country_code_a3', 'tab':'sector', 'sub-region_name':'subregion_name'}, inplace=True)
df_final_energy.to_csv('../../../data/_processed/iea_final_energy.csv', index=False)
df_final_energy.head()

,country,country_code_a3,year,sector,region_name,subregion_name,unit,value,source,type
0,Albania,ALB,2000,Industry and services,Europe,Southern Europe,PJ,25.12,iea,final_energy
1,Albania,ALB,2000,Passenger transport,Europe,Southern Europe,PJ,20.14,iea,final_energy
2,Albania,ALB,2000,Residential,Europe,Southern Europe,PJ,15.75,iea,final_energy
3,Albania,ALB,2001,Industry and services,Europe,Southern Europe,PJ,25.65,iea,final_energy
4,Albania,ALB,2001,Passenger transport,Europe,Southern Europe,PJ,21.17,iea,final_energy


### SHARE_COUNTRY_EM

In [47]:
df_share_country_em = df_iea[df_iea['flow']=='SHARE_COUNTRY_EM']
df_share_country_em.head()

,country,tab,flow,enduse,activity,unit,year,value
69,Australia,Freight transport,SHARE_COUNTRY_EM,Freight trains,NaN,%,2013,0.90
70,Australia,Freight transport,SHARE_COUNTRY_EM,Trucks,NaN,%,2013,8.65
83,Australia,Freight transport,SHARE_COUNTRY_EM,Domestic freight ships,NaN,%,2014,0.19
84,Australia,Freight transport,SHARE_COUNTRY_EM,Freight trains,NaN,%,2014,0.92
85,Australia,Freight transport,SHARE_COUNTRY_EM,Trucks,NaN,%,2014,8.99
